Big Data Analytics Techniques and Applications HW2

In [ ]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jre-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import os
import glob
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, lit, coalesce, greatest

drive.mount('/content/gdrive')
# files = glob.glob(os.path.join('/content/gdrive/My Drive/bigdata/hw2', '*.csv'))
path = '/content/gdrive/My Drive/bigdata/hw2/'

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .getOrCreate() 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Q1. Find the maximal delays (you should consider both ArrDelay and DepDelay) for each month of 2007.**

In [ ]:
file = path + '2007.csv'
df = spark.read.csv(file, header=True)
# print(df.count())
df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

In [ ]:
df = df.withColumn('ArrDelay', df['ArrDelay'].cast(IntegerType()))
df = df.withColumn('DepDelay', df['DepDelay'].cast(IntegerType()))

df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDela

In [ ]:
dfDelay = df.groupby('Month').agg(F.max('ArrDelay').alias('MaxArrDelay'), \
                  F.max('DepDelay').alias('MaxDepDelay'))
minf = lit(float("-inf"))
delaymax = greatest(*[coalesce(col(x), minf) for x in ['MaxArrDelay', 'MaxDepDelay']])
dfDelay.withColumn("MaxDelay", delaymax).show()

+-----+-----------+-----------+--------+
|Month|MaxArrDelay|MaxDepDelay|MaxDelay|
+-----+-----------+-----------+--------+
|    3|       1564|       1547|  1564.0|
|    1|       1426|       1406|  1426.0|
|    2|       1359|       1340|  1359.0|
|    5|       1429|       1416|  1429.0|
|    4|       1402|       1415|  1415.0|
|    7|       1386|       1369|  1386.0|
|    6|       1351|       1360|  1360.0|
|    8|       1472|       1449|  1472.0|
|    9|       1665|       1689|  1689.0|
|   10|       2598|       2601|  2601.0|
|   11|       1146|       1137|  1146.0|
|   12|       1942|       1956|  1956.0|
+-----+-----------+-----------+--------+



In [ ]:
# for m in range(1, 13):
#     new_df = df.filter(df['Month'] == str(m))
#     max_arrdelay = new_df.agg({"ArrDelay": "max"}).collect()[0][0]
#     max_depdelay = new_df.agg({"DepDelay": "max"}).collect()[0][0]

#     maxDelay = max(max_arrdelay, max_depdelay)

#     print('2017/{} MaxDelay:{}, with ArrDelay:{} and DepDelay:{}'.format(m, maxDelay, max_arrdelay, max_depdelay))

**Q2. How many flights were delayed caused by security between 2000 ~ 2005? Please show the counting for each year.**

In [ ]:
files = []
for i in range(2000, 2005+1):
    files.append(path + str(i) + '.csv')
print(files)
df = spark.read.csv(files, header=True)

['/content/gdrive/My Drive/bigdata/hw2/2000.csv', '/content/gdrive/My Drive/bigdata/hw2/2001.csv', '/content/gdrive/My Drive/bigdata/hw2/2002.csv', '/content/gdrive/My Drive/bigdata/hw2/2003.csv', '/content/gdrive/My Drive/bigdata/hw2/2004.csv', '/content/gdrive/My Drive/bigdata/hw2/2005.csv']


In [ ]:
df = df.withColumn('SecurityDelay', df['SecurityDelay'].cast(IntegerType()))
new_column = when(col('SecurityDelay') > 0, 1).otherwise(0)

df = df.withColumn('HasSecDelay', new_column)
df.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+-----------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|HasSecDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+-----------+
|2000|    1|        28|        5|   1647|      1647|   190

In [ ]:
df.groupby('Year').agg({'HasSecDelay': 'sum'}).show()

+----+----------------+
|Year|sum(HasSecDelay)|
+----+----------------+
|2000|               0|
|2001|               0|
|2002|               0|
|2003|            3740|
|2004|            8158|
|2005|            6627|
+----+----------------+



Q3.  List Top 5 airports which occur delays most and least in 2008. (Please show the IATA airport code)

In [ ]:
file = path + '2008.csv'
df = spark.read.csv(file, header=True)
df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

In [ ]:
df = df.withColumn('ArrDelay', df['ArrDelay'].cast(IntegerType()))
df = df.withColumn('DepDelay', df['DepDelay'].cast(IntegerType()))

new_column = when((col('ArrDelay') > 0) | \
                  (col('DepDelay') > 0), 1).otherwise(0)

df1 = df.withColumn('HasDelay', new_column)
df1.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+--------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|HasDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+--------+
|2008|    1|         3|        4|   2003|      1955|   2211|      2

In [ ]:
df2 = df1.groupby('Origin').agg(F.sum('HasDelay').alias('Count'))
df2.orderBy('Count', ascending=False).show(10)
df2.orderBy('Count').show(10)

+------+------+
|Origin| Count|
+------+------+
|   ATL|237091|
|   ORD|197937|
|   DFW|160519|
|   DEN|134941|
|   LAX|112426|
|   IAH|109517|
|   PHX|105595|
|   LAS| 92915|
|   EWR| 89004|
|   DTW| 88197|
+------+------+
only showing top 10 rows

+------+-----+
|Origin|Count|
+------+-----+
|   PUB|    2|
|   TUP|    3|
|   PIR|    4|
|   INL|   11|
|   BJI|   12|
|   BLI|   18|
|   HTS|   23|
|   WYS|   29|
|   SUX|   38|
|   ACY|   52|
+------+-----+
only showing top 10 rows

